In [4]:
import os
import re
import json
import pandas as pd
import numpy as np
from functools import partial
from IPython.core.debugger import set_trace
from tqdm import tqdm_notebook
#from multiprocessing import Pool, Process
from ckonlpy.tag import Twitter
import json
import ijson
from gensim.models import Word2Vec, FastText

In [3]:
with open('twitter_data/qry.json') as f:
    qry = json.load(f)
    
new_words = list(set(sum([re.findall(r'#([^ ]+)', v)+[k] for k,v in qry.items()], [])))

In [4]:
new_words

['eider',
 '몽벨',
 'yoshidaporter',
 'yojiyamamoto',
 'dailypaper',
 'jwanderson',
 '카피탈',
 'prada',
 'barrel',
 'dynafit',
 'ragandbone',
 'canadagoose',
 'abecrombiefitch',
 '몽클레르',
 '아디다스오리지널',
 'incase',
 'helmutlang',
 'pearlygates',
 '99퍼센트이즈',
 '걸스돈크라이',
 'undftd',
 'brixton',
 'saintlaurent',
 'cherryla',
 '스포츠웨어',
 'poloralphlauren',
 'stoneisland',
 'thursdayisland',
 'charms',
 'lafuma',
 '헤리티지플로스',
 '의류',
 'doublet',
 'bally',
 'gucci',
 '와이쓰리',
 'cutlerandgross',
 '미유미유',
 'pleatsplease',
 '니들스',
 'needles',
 'plac',
 'hagt',
 '이자벨마랑',
 'columbia',
 '폴로랄프로렌',
 '잔스포츠',
 '발망',
 '제이린드버그',
 '버켄스탁',
 '에이프',
 '미즈노',
 '버커루',
 '컬럼비아',
 'cpcompany',
 'abecrombie',
 '생로랑',
 'streetwear',
 'nerdy',
 'ferragamo',
 '뉴발란스',
 '인케이스',
 '닥터마틴',
 '레이닝챔프',
 '레디메이드',
 '젠틀몬스터',
 'yoshida',
 'readymade',
 '페레가모',
 '베자',
 'tods',
 '앰씨앰',
 '디스커버리',
 'salvatoreferregamo',
 '앨앰씨',
 'plasticisland',
 'soup',
 '비츠바이닥터드레',
 '디키즈',
 'vejasneakers',
 '허프',
 'series',
 '참스',
 '톰스슈즈',
 '옷',
 '스포츠패션',
 'mun

In [6]:
twitter = Twitter()
twitter.add_dictionary(new_words, 'Noun')

In [8]:
srcdir = 'twitter_data'
textcol = 'text'
fnames = [srcdir + '/' + fname for fname in os.listdir(srcdir)];fnames

['twitter_data/032c (2010.01.01-2019.01.31).pkl',
 'twitter_data/1017alyx9sm (2010.01.01-2019.01.31).pkl',
 'twitter_data/8seconds (2010.01.01-2019.01.31).pkl',
 'twitter_data/99percentis (2010.01.01-2019.01.31).pkl',
 'twitter_data/a.p.c (2010.01.01-2019.01.31).pkl',
 'twitter_data/aape (2010.01.01-2019.01.31).pkl',
 'twitter_data/abecrombiefitch (2010.01.01-2019.01.31).pkl',
 'twitter_data/acnestudios (2010.01.01-2019.01.31).pkl',
 'twitter_data/acoldwall (2010.01.01-2019.01.31).pkl',
 'twitter_data/acronym (2010.01.01-2019.01.31).pkl',
 'twitter_data/adererror (2010.01.01-2019.01.31).pkl',
 'twitter_data/adidas (2010.01.01-2019.01.31).pkl',
 'twitter_data/adidasoriginal (2010.01.01-2019.01.31).pkl',
 'twitter_data/advisoryboardcrystals (2010.01.01-2019.01.31).pkl',
 'twitter_data/aecawhite (2010.01.01-2019.01.31).pkl',
 'twitter_data/aimeleondore (2010.01.01-2019.01.31).pkl',
 'twitter_data/aldo (2010.01.01-2019.01.31).pkl',
 'twitter_data/alexandermcqueen (2010.01.01-2019.01.31).pk

In [9]:
tokens_dict = {}
for fname in tqdm_notebook(fnames):
    try:
        item, tokenized = tokenize(fname, textcol)
        tokens_dict[item] = tokenized
    except:
        pass

In [10]:
with open('twitter_data/tokens_dict.json', 'w') as f:
    json.dump(tokens_dict, f)

In [5]:
with open('twitter_data/tokens_dict.json') as f:
    tokens_dict = json.load(f)

# Word2Vec: 잘됨

In [27]:
w2v = Word2Vec(sum(tokens_dict.values(), []), size=100, window=5, min_count=10, workers=4, sg=1)
w2v.save('twitter_data/word2vec.model')

In [28]:
w2v.init_sims(replace=True)

In [29]:
w2v = Word2Vec.load('twitter_data/word2vec.model')

In [57]:
w2v.wv.most_similar(positive=['저렴'], topn=10)

[('김민선', 0.6446695923805237),
 ('13843337953', 0.6186510324478149),
 ('가격', 0.6076311469078064),
 ('한데', 0.6006898880004883),
 ('115', 0.596698522567749),
 ('99000', 0.5779435634613037),
 ('149.5', 0.577297568321228),
 ('착하다', 0.5686609745025635),
 ('만족도', 0.5636894106864929),
 ('콰', 0.5629968643188477)]

# FastText는 한글에서 잘 안먹히는 것 같다

In [32]:
sentences = sum(tokens_dict.values(), [])
fasttext = FastText(size=100, window=5, min_count=10)
fasttext.build_vocab(sentences=sentences)
fasttext.train(sentences=sentences, total_examples=len(sentences), epochs=10)
fasttext.save('twitter_data/fasttext.model')

In [33]:
fasttext.init_sims(replace=True)

In [34]:
fasttext = FastText.load('twitter_data/fasttext.model')

In [35]:
fasttext.wv.most_similar(positive=['나이키'], topn=10)

[('이키', 0.7119917869567871),
 ('나이테', 0.6592456102371216),
 ('나이트런', 0.581122636795044),
 ('sb', 0.5587837100028992),
 ('ナイキ', 0.557768702507019),
 ('에어', 0.5537333488464355),
 ('gyakusou', 0.5514733791351318),
 ('머큐리얼', 0.5435261726379395),
 ('덩크', 0.5427482724189758),
 ('포짓', 0.5412664413452148)]

In [8]:
class Tokens:
    def __init__(self, tokens_dict):
        self.tokens_dict = tokens_dict
        self.length = 0
        
    def __iter__(self):
        for _, v in self.tokens_dict.items():
            yield from v

    def __len__(self):
        if self.length == 0:
            for fname in self.fnames:
                self.length += len(pd.read_pickle(fname))
            
        return self.length

In [18]:
l = ijson.items(f, 'crocs.item')

In [7]:
def preproc(text, remove_url=True, remove_mention=True, remove_hashtag=False):
    LINEBREAK = r'\n' # str.replace에서는 r'\n'으로 검색이 안된다
    RT = ' rt '
    EMOJI = r'[\U00010000-\U0010ffff]'
    DOTS = '…'
    LONG_BLANK = r'[ ]+'
    
    # \u3131-\u3163\uac00-\ud7a3 는 한글을 의미함
    # URL = r'(?P<url>(https?://)?(www[.])?[^ \u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b([^ \u3131-\u3163\uac00-\ud7a3]*))'
    URL1 = r'(?:https?:\/\/)?(?:www[.])?[^ :\u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b(?:[^ \u3131-\u3163\uac00-\ud7a3]*)'
    URL2 = r'pic.twitter.com/[a-zA-Z0-9_]+'
    URL = '|'.join((URL1, URL2))
    
    HASHTAG = r'#(?P<inner_hashtag>[^ #@]+)'
    MENTION = r'@(?P<inner_mention>[^ #@]+)' 
    
    #PTNS = '|'.join((LINEBREAK, RT, URL, HASHTAG, MENTION, EMOJI))
    
    #out = {}
    text = re.sub('|'.join((LINEBREAK, RT, EMOJI, DOTS)), '', text.lower())
    
    if remove_url:
        text = re.sub(URL, '', text)

    if remove_mention:
        text = re.sub(MENTION, '', text)        
    else:
        text = re.sub(MENTION, ' \g<inner_mention>', text)
        
    if remove_hashtag:
        text = re.sub(HASHTAG, '', text)        
    else:
        text = re.sub(HASHTAG, ' \g<inner_hashtag>', text)
        
    return re.sub(LONG_BLANK, ' ', text).strip()


def tokenize(fname, textcol):
    df = pd.read_pickle(fname)
    item = df.columns.levels[0][0]
    tokenized = []
    
    for doc in df.droplevel(0, axis=1)[textcol]:
        text = preproc(doc)
        _tokenized = [t[0] for t in twitter.pos(text, norm=True, stem=True) if t[1] not in ['Punctuation', 'Josa']]
        tokenized.append(_tokenized)
    
    return item, tokenized

In [10]:
preproc(pd.read_pickle('twitter_data/allsaints (2010.01.01-2019.02.07).pkl').droplevel(0,axis=1).text.iloc[210], remove=True)

{'urls': ['http://www.fashionn.com/board/read_new.php?table=1006&number=21030&sns_number=989523…',
  'pic.twitter.com/2rys36ronu'],
 'hashtags': ['올세인츠', '올세인츠비치컬렉션pic.twitter.com/2rys36ronu'],
 'mention': [],
 'text': "올세인츠,  여름 한정판 '헌팅턴 비치 컬렉션' 출시"}

In [146]:
preproc(pd.read_pickle('twitter_data/allsaints (2010.01.01-2019.02.07).pkl').droplevel(0,axis=1).text.iloc[42])

'수퍼스타 커플신발 신세계백화점강남점 시댁선물사러나왔다가 allsaints 거울이좋네 ㅋㅋㅋ 얼마만에 같이나온건지~ 데이트하는기분'

In [145]:
pd.read_pickle('twitter_data/allsaints (2010.01.01-2019.02.07).pkl').droplevel(0,axis=1).text[42]

'#수퍼스타#커플신발#신세계백화점강남점 #시댁선물사러나왔다가 #ALLSAINTS #거울이좋네 ㅋㅋㅋ#얼마만에 같이나온건지~#데이트하는기분 https://www.instagram.com/p/-8DHERpvz-/\xa0'